# Sheikhspeare

## AI chatbot that speaks in Shakespearean English

<a href="https://colab.research.google.com/github/cybardev/Sheikhspeare/blob/main/sheikhspeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TODO: install deps